In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [4]:
val sameModel = PipelineModel.load(modelPath)

sameModel = pipeline_143427c788bd


pipeline_143427c788bd

In [5]:
val predictionsDF = sameModel.transform(inputDF)



predictionsDF = [tweet: string, words: array<string> ... 5 more fields]


[tweet: string, words: array<string> ... 5 more fields]

In [6]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [7]:
predictionsDF.printSchema()

root
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)
 |-- predictedLabel: string (nullable = true)



In [8]:

predictionsDF
    .select($"tweet", $"words", $"features", $"rawPrediction", $"probability", $"prediction", getProbability($"probability").alias("clean_probability"))
    .writeStream
    .foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@a590c

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction| clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------+
|@thegame123 ohw  ...|[@thegame123, ohw...|(1000,[372,594,60...|[4.87917110966780...|[0.48791711096678...|       1.0| 0.512082889033219|
|Wow .. last night...|[wow, .., last, n...|(1000,[36,61,76,1...|[4.93721820970158...|[0.49372182097015...|       1.0|0.5062781790298415|
|Have a stupid res...|[have, a, stupid,...|(1000,[72,86,121,...|[5.27186922549893...|[0.52718692254989...|       0.0|0.4728130774501067|
|@zombietwitch I l...|[@zombietwitch, i...|(1000,[239,263,29...|[4.75012512683551...|[0.47501251268355...|       1.0|0.5249874873164485|
|@msmcporkchop wha...|[@msmcporkchop, w..

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|britains got tale...|[britains, got, t...|(1000,[7,97,183,3...|[5.39843146807782...|[0.53984314680778...|       0.0|0.46015685319221716|
|No, I was waving ...|[no,, i, was, wav...|(1000,[91,130,234...|[5.04148782637064...|[0.50414878263706...|       0.0|0.49585121736293597|
|@pscott22 i watch...|[@pscott22, i, wa...|(1000,[11,128,170...|[4.93436105532784...|[0.49343610553278...|       1.0|  0.506563894467216|
|True friend - som...|[true, friend, -,...|(1000,[118,188,21...|[4.43604348137964...|[0.44360434813796...|       1.0| 0.5563956518620358|
|Essa internet t� ...|[essa, inter

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|instead of enjoyi...|[instead, of, enj...|(1000,[240,343,35...|[4.71187360950076...|[0.47118736095007...|       1.0| 0.5288126390499235|
|Feeling like crap...|[feeling, like, c...|(1000,[5,162,282,...|[4.97186463452649...|[0.49718646345264...|       1.0| 0.5028135365473505|
|@jennysunphoto ye...|[@jennysunphoto, ...|(1000,[44,346,437...|[5.84321573162323...|[0.58432157316232...|       0.0|0.41567842683767636|
|ehh I need to cle...|[ehh, i, need, to...|(1000,[329,340,38...|[5.63794180065782...|[0.56379418006578...|       0.0|  0.436205819934217|
|@MsNanai thanks f...|[@msnanai, t

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|Aww nobody likes ...|[aww, nobody, lik...|(1000,[77,170,318...|[4.84456570320690...|[0.48445657032069...|       1.0| 0.5155434296793093|
|@Fearnecotton omg...|[@fearnecotton, o...|(1000,[36,44,100,...|[4.96896646178874...|[0.49689664617887...|       1.0| 0.5031033538211256|
|Who uses their we...|[who, uses, their...|(1000,[111,217,23...|[5.38970546744485...|[0.53897054674448...|       0.0|0.46102945325551437|
|Missing kickball ...|[missing, kickbal...|(1000,[65,115,168...|[4.97237511110869...|[0.49723751111086...|       1.0| 0.5027624888891303|
|Can't access USPS...|[can't, acce

...

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|My back hurts lik...|[my, back, hurts,...|(1000,[81,196,239...|[5.71396386755535...|[0.57139638675553...|       0.0| 0.4286036132444641|
|Birthday so far.....|[birthday, so, fa...|(1000,[12,36,39,7...|[5.65307730652381...|[0.56530773065238...|       0.0|0.43469226934761807|
|Working on the ph...|[working, on, the...|(1000,[23,36,82,1...|[4.96896646178874...|[0.49689664617887...|       1.0| 0.5031033538211256|
|Yuk, I'm poorly t...|[yuk,, i'm, poorl...|(1000,[36,155,388...|[5.03626564394731...|[0.50362656439473...|       0.0| 0.4963734356052683|
|Woke up late and ...|[woke, up, l

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction| clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------+
|still searching a...|[still, searching...|(1000,[36,168,170...|[4.64949514657854...|[0.46494951465785...|       1.0|0.5350504853421458|
|@alisonmf your IS...|[@alisonmf, your,...|(1000,[114,117,16...|[4.65908732283917...|[0.46590873228391...|       1.0|0.5340912677160823|
|@NathanFillion Aw...|[@nathanfillion, ...|(1000,[37,86,187,...|[5.19054790805622...|[0.51905479080562...|       0.0|0.4809452091943774|
|imz so bored  not...|[imz, so, bored, ...|(1000,[6,18,146,1...|[4.96896646178874...|[0.49689664617887...|       1.0|0.5031033538211256|
|@ShellyKramer Eh,...|[@shellykramer, e..

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|You were my first...|[you, were, my, f...|(1000,[13,18,36,3...|[4.53904695488118...|[0.45390469548811...|       1.0| 0.5460953045118815|
|'I'm so 2008, you...|['i'm, so, 2008,,...|(1000,[0,82,96,99...|[4.83992047895644...|[0.48399204789564...|       1.0| 0.5160079521043551|
|i´m afraid of the...|[i´m, afraid, of,...|(1000,[23,30,36,4...|[4.93725922806559...|[0.49372592280655...|       1.0| 0.5062740771934406|
|@maytjh SEVEN hou...|[@maytjh, seven, ...|(1000,[42,159,195...|[4.87917110966780...|[0.48791711096678...|       1.0|  0.512082889033219|
|left my cat out i...|[left, my, c

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
| cant afford to s...|[, cant, afford, ...|(1000,[26,36,197,...|[5.03332455510581...|[0.50333245551058...|       0.0| 0.4966675444894186|
|@FranklinDavid an...|[@franklindavid, ...|(1000,[19,147,150...|[4.62635843309537...|[0.46263584330953...|       1.0| 0.5373641566904624|
|@soonseeofpred be...|[@soonseeofpred, ...|(1000,[425,537,78...|[4.56234815445403...|[0.45623481544540...|       1.0| 0.5437651845545968|
|Does anybody have...|[does, anybody, h...|(1000,[36,48,80,8...|[5.05697352045977...|[0.50569735204597...|       0.0| 0.4943026479540224|
|Someones boat jus...|[someones, b

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|@trips05 u abando...|[@trips05, u, aba...|(1000,[36,122,240...|[4.83543868490889...|[0.48354386849088...|       1.0| 0.5164561315091105|
|@SongzYuuup VA lo...|[@songzyuuup, va,...|(1000,[34,81,82,8...|[5.09862269540705...|[0.50986226954070...|       0.0|  0.490137730459295|
|@ofallpieces Oooo...|[@ofallpieces, oo...|(1000,[71,329,566...|[5.51517527406463...|[0.55151752740646...|       0.0| 0.4484824725935364|
|I thought, for a ...|[i, thought,, for...|(1000,[7,36,82,96...|[4.39637962106026...|[0.43963796210602...|       1.0| 0.5603620378939735|
|@BlueLint Good po...|[@bluelint, 